In [7]:
import datasets

dataset = datasets.load_dataset("philschmid/markdown-documentation-transformers")

dataset

df = dataset["train"].to_pandas()

df.head()

,title,url,markdown,html,crawlDate
0,🤗 Transformers,https://huggingface.co/docs/transformers/v4.34...,# 🤗 Transformers\n\nState-of-the-art Machine L...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:32.994Z
1,Quick tour,https://huggingface.co/docs/transformers/v4.34...,# Quick tour\n\nGet up and running with 🤗 Tran...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:34.381Z
2,Installation,https://huggingface.co/docs/transformers/v4.34...,# Installation\n\nInstall 🤗 Transformers for w...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:34.498Z
3,Pipelines for inference,https://huggingface.co/docs/transformers/v4.34...,# Pipelines for inference\n\nThe [pipeline()](...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:35.128Z
4,Load pretrained instances with an AutoClass,https://huggingface.co/docs/transformers/v4.34...,# Load pretrained instances with an AutoClass\...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:35.231Z


In [8]:
df = df.rename(columns={'markdown': 'context'})
df.head()

,title,url,context,html,crawlDate
0,🤗 Transformers,https://huggingface.co/docs/transformers/v4.34...,# 🤗 Transformers\n\nState-of-the-art Machine L...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:32.994Z
1,Quick tour,https://huggingface.co/docs/transformers/v4.34...,# Quick tour\n\nGet up and running with 🤗 Tran...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:34.381Z
2,Installation,https://huggingface.co/docs/transformers/v4.34...,# Installation\n\nInstall 🤗 Transformers for w...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:34.498Z
3,Pipelines for inference,https://huggingface.co/docs/transformers/v4.34...,# Pipelines for inference\n\nThe [pipeline()](...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:35.128Z
4,Load pretrained instances with an AutoClass,https://huggingface.co/docs/transformers/v4.34...,# Load pretrained instances with an AutoClass\...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:35.231Z


In [9]:
from counter import get_and_increment_counter
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.memory_insert_params import Document

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

providers = client.providers.list()
memory_banks_response = client.memory_banks.list()

bank_id = f"bank_{get_and_increment_counter()}"
provider = providers["memory"][0]
client.memory_banks.register(
    memory_bank_id=bank_id,
    params={
        "embedding_model": "all-MiniLM-L6-v2",
        # Is the default for agent config: https://github.com/meta-llama/llama-stack/blob/66d8f4ffd126bff668434b314892a99fe854a034/llama_stack/providers/inline/agents/meta_reference/agent_instance.py#L668
        "chunk_size_in_tokens": 512,
    },
    provider_id=provider.provider_id,
)
bank_id

'bank_53'

In [10]:
documents = [
    Document(
        document_id=str(idx),
        content=context,
        mime_type="text/plain",
        metadata={},
    )
    for idx, context in zip(df.index, df["context"])
]
documents[:5]

[{'document_id': '0',
  'content': '# 🤗 Transformers\n\nState-of-the-art Machine Learning for [PyTorch](https://pytorch.org/), [TensorFlow](https://www.tensorflow.org/), and [JAX](https://jax.readthedocs.io/en/latest/).\n\n🤗 Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. These models support common tasks in different modalities, such as:\n\n📝 **Natural Language Processing**: text classification, named entity recognition, question answering, language modeling, summarization, translation, multiple choice, and text generation.  \n🖼️ **Computer Vision**: image classification, object detection, and segmentation.  \n🗣️ **Audio**: automatic speech recognition and audio classification.  \n🐙 **Multimodal**: table question answering, optical character recognition, information extraction from sca

In [11]:
from tqdm import tqdm

for i in tqdm(range(len(documents))):
    client.memory.insert(
        bank_id=bank_id,
        documents=[documents[i]],
    )

100%|███████████████████████████████████████████████████████████████████████| 386/386 [1:05:43<00:00, 10.22s/it]


 31%|██████████████████████▋                                                  | 120/386 [06:39<35:29,  8.01s/it]